# 1|Importıng Modules

In [ ]:
import os
import random
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.preprocessing import image
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import cv2

# 2|EDA

In [ ]:

test_set =  "Dataset/test"
train_set = "Dataset/train"
val_set =   "Dataset/val"

class_names = os.listdir(train_set)
sizes = [len(os.listdir(train_set + '/' + name)) for name in class_names]
sizes

In [ ]:
fig = px.pie(
         names=class_names,
         values=sizes,
         width = 450,
         title='Class Distribution',
         hole=0.4
)
fig.update_layout({'title':{'x':0.45}})
fig.show()

In [5]:
def load_random_img(dir,label):
     plt.figure(figsize=(10,10))
     i=0
     for label in class_names:
      i+=1
      plt.subplot(1,5,i)
      file=random.choice(os.listdir(dir+'/test/'+label))
      image_path = os.path.join(dir+'/test/'+label, file)
      print(image_path)

      img=cv2.imread(image_path)
      plt.title(label)
      plt.imshow(img)
      plt.grid(None)
      plt.axis('off')


In [ ]:
for i in range(3):
    load_random_img("Dataset", class_names)

# 3|Data Spliting

In [8]:
train_datagen = image.ImageDataGenerator(
    rotation_range=15,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)
validation_datagen= image.ImageDataGenerator(    rotation_range=15,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1)
test_datagen= image.ImageDataGenerator(    rotation_range=15,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1)


In [ ]:
# image addressing
train_generator = train_datagen.flow_from_directory(
    train_set,
    target_size = (224,224),
    batch_size = 16,
    class_mode = 'categorical')
validation_generator = validation_datagen.flow_from_directory(
    val_set,
    target_size = (224,224),
    batch_size = 16,
    shuffle=True,
    class_mode = 'categorical')
test_generator = test_datagen.flow_from_directory(
    test_set,
    target_size = (224,224),
    batch_size = 16,
    class_mode = 'categorical')

In [ ]:
train_generator.class_indices

# 4|CNN MODEL

In [ ]:
from tensorflow.keras import regularizers
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Input, GaussianNoise, GlobalAveragePooling2D, Dense, BatchNormalization, Dropout
from tensorflow.keras.models import Sequential, Model
import tensorflow as tf


base_model = tf.keras.applications.EfficientNetB2(weights='imagenet', input_shape=(224,224,3), include_top=False)

for layer in base_model.layers:
    layer.trainable=False

# Define the input layer explicitly
input_tensor = Input(shape=(224, 224, 3))

# Pass the input tensor to the base model
x = base_model(input_tensor, training=False)  # Set training=False for inference

# Add the remaining layers
x = GaussianNoise(0.35)(x)
x = GlobalAveragePooling2D()(x)
x = Dense(256,activation='relu')(x)
x = BatchNormalization()(x)
x = GaussianNoise(0.35)(x)
x = Dropout(0.2)(x)
output_tensor = Dense(4, activation='softmax')(x)

# Create the final model
model = Model(inputs=input_tensor, outputs=output_tensor)

model.summary()


In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy','Precision','Recall','AUC'])
model.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy','Precision','Recall','AUC'])


In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=20, #steps_per_epoch= training_images/batch_size= 3200/32<=200
    epochs = 20, #own choice
    validation_data = validation_generator
)